In [1]:
import numpy as np
import pandas as pd
import math

import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib widget
import itertools
import warnings
warnings.filterwarnings('ignore')

### Aggregated Mean Wind Speed Time Series
Every sample is the average wind speed across all turbines

In [2]:
colab_path = 'ScadaData.txt'
local_path = 'C:\\Users\\hanna\\Desktop\\data\\ScadaData.txt'

scada_data = pd.read_csv(local_path, delimiter='\t',  parse_dates=True)
scada_data['dtTimeStamp'] = pd.to_datetime(scada_data['dtTimeStamp'])
average_mean_wind_speeds = scada_data[['dtTimeStamp','WindSpeedMean', 'intObjectId']].groupby('dtTimeStamp').mean()[['WindSpeedMean']]

print("Number of turbines: " + str(len(scada_data['intObjectId'].unique())))
print(average_mean_wind_speeds.head(5))

Number of turbines: 27
                     WindSpeedMean
dtTimeStamp                       
2019-08-01 00:00:00       2.677778
2019-08-01 00:10:00       2.914815
2019-08-01 00:20:00       3.700000
2019-08-01 00:30:00       4.488889
2019-08-01 00:40:00       4.977778


### Raw Data Plot

In [3]:
plt.figure(figsize=(20,5))
plt.plot(average_mean_wind_speeds["WindSpeedMean"], color='blue', label='seasonal')
plt.legend(loc='best')
plt.title('Seasonal Component')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Seasonal Component')

### Forecasting 1 time step
Forecasting the 10 minutes and then retraining with the real value for the forecast and repeating

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import datetime as dt

X = average_mean_wind_speeds[0:2000]
size = int(len(X) * 0.04)
train, test = X[0:size], X[size:len(X)]
history = train
predictions = pd.DataFrame(index=test.index, columns=['prediction'])
times = []
for i, row in test.iterrows():
    start = dt.datetime.now()
    model = ARIMA(history, order=(1,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0].flatten()[0]
    predictions.loc[i]['prediction'] = yhat
    history.loc[i] = row['WindSpeedMean']
    end = dt.datetime.now()
    times.append(end-start)
    #print('predicted=%f, expected=%f' % (predictions.loc[i]['prediction'], row['WindSpeedMean']))

print('done')
print((sum(times, dt.timedelta()) / len(times)))

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(history, color='blue', label='real')
plt.plot(predictions, color='red', label='prediction')
plt.legend(loc='best')
plt.title('Real vs Predictions')

In [ ]:
combined_df = pd.concat([history,predictions], axis=1).dropna()
print(combined_df)

In [ ]:
import random
import numpy
from matplotlib import pyplot

plt.hist([combined_df['WindSpeedMean'], combined_df['prediction']], bins=list(range(0,15)), label=['Real', 'Prediction'])
plt.legend(loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(combined_df['WindSpeedMean'], combined_df['prediction'])
print('Test MSE: %.3f' % error)

### Forecasting 1 hour

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import datetime as dt


n_step = 6
X = average_mean_wind_speeds[0:2000]
size = int(len(X) * 0.04)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train.values]
predictions = list()
times = []
test_indices = [i for i in range(0, len(test.values)) if i%n_step==0]
for i in test_indices:
    start = dt.datetime.now()
    model = ARIMA(history, order=(1,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast(n_step)
    yhat = output[0].flatten()
    predictions.extend(yhat)
    history.extend(test.values[i:i+n_step])
    end = dt.datetime.now()
    times.append(end-start)
    #print('predicted=%f, expected=%f' % (predictions.loc[i]['prediction'], row['WindSpeedMean']))

print('done')
print((sum(times, dt.timedelta()) / len(times)))


In [ ]:
print(len(predictions))

predictions_df = pd.DataFrame(index=test.index, columns=['Predictions'])
predictions_df['Predictions'] = predictions

# print('done')

plt.figure(figsize=(20,5))
plt.plot(X, color='blue', label='real')
plt.plot(predictions_df, color='red', label='prediction')
plt.legend(loc='best')
plt.title('Real vs Predictions')

In [ ]:
import random
import numpy
from matplotlib import pyplot

combined_df = pd.concat([X,predictions_df], axis=1).dropna()
print(combined_df)

plt.hist([combined_df['WindSpeedMean'], combined_df['Predictions']], bins=list(range(0,15)), label=['Real', 'Prediction'])
plt.legend(loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
error = mean_squared_error(combined_df['WindSpeedMean'], combined_df['Predictions'])
print('Test MSE: %.3f' % error)